In [2]:
from data import get_hf_image_dataset, image_transform
from model import UIL

In [3]:
model_config = dict(
        image_size=224,
        patch_size=16,
        width=768,
        layers=12,
        heads=12,
        noise_std=0.8,
        mask_ratio=0.75,
        decoder_layers=8,
        decoder_width=512,
        decoder_heads=16,
        dropout_rate=0,
        attn_dropout_rate=0,
        do_denoise=True,
        do_mae=True,
        do_causal=True,
    )

model = UIL(**model_config, deterministic=True)